### Made different notebooks for fine tuning , this is for model comparison

Notebook on Finetuning Llama --> https://www.kaggle.com/code/piyushguptav/fine-tune-llama

Notebook on Finetuning Mistral --> https://www.kaggle.com/code/piyushguptav/fine-tune-llama

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [10]:
import pandas as pd
df = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')
df.drop('id', axis=1, inplace=True)
df.head()

,article,highlights
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     13368 non-null  object
 1   highlights  13368 non-null  object
dtypes: object(2)
memory usage: 209.0+ KB


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
!pip install rouge-score nltk tqdm bert_score evaluate

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'rouge-score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge-score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6b19e9d939b1d48b7f0279580c54d29d56f2b7b179bb62f71481714a7b5c17f5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bert_score]


In [5]:
!pip install --upgrade pip
!pip install torch==2.5.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu124
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-xqqsozqr/unsloth_9a8916a6403042d7b5aa01703337219f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-xqqsozqr/unsloth_9a8916a6403042d7b5aa01703337219f
  Resolved https://github.com/unslothai/unsloth.git to commit e596938ccf3750e153e7105f8123f94e0823d1e7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu124/xformers-0.0.28.post2-cp311-cp311-manylinux_2_28_x86_64.whl (20.6 MB)


In [12]:
df_fifty = df.sample(n=50, random_state=42).reset_index(drop=True)

In [13]:
# === Load Data (only 50 samples) ===

input_texts = df_fifty["article"].tolist()
reference_summaries = df_fifty["highlights"].tolist()

In [7]:
import torch, re, nltk
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score
import evaluate

# Download NLTK models
nltk.download("punkt")

# === BitsAndBytes 4-bit quantization config for Mistral ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
    bnb_4bit_compute_dtype=torch.float16
)

# === Load Model A (Mistral) in 4-bit on GPU with offloading ===
model_a = AutoModelForCausalLM.from_pretrained(
    "badbrock/mistral-7b-finetuned",
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer_a = AutoTokenizer.from_pretrained("badbrock/mistral-7b-finetuned")
model_b = AutoModelForCausalLM.from_pretrained(
    "badbrock/llama3-1b-lora-summary",
    quantization_config=bnb_config,
    device_map="auto"
)
# === Load Model B on CPU ===
tokenizer_b = AutoTokenizer.from_pretrained("badbrock/llama3-1b-lora-summary")

2025-05-14 07:03:27.008654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747206207.226421      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747206207.292649      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


adapter_config.json:   0%|          | 0.00/797 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [8]:
# === Ensure tokenizer is set up properly ===
def prepare_model(model, tokenizer):
    if tokenizer.eos_token is None:
        tokenizer.eos_token = "</s>"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return model

prepare_model(model_a, tokenizer_a)
prepare_model(model_b, tokenizer_b)

# === Prompt Template ===
def build_prompt(script_2):
    return f"""<|im_start|>system
You are a summarization AI. Generate only a 2-3 sentence summary. Always complete your sentences.
<|im_end|>
<|im_start|>user
Summarize this article in 2-3 complete sentences:

{script_2}
<|im_end|>
<|im_start|>assistant
"""

# === Clean Summary Output ===
def clean_summary(summary, prompt_ids, tokenizer):
    decoded = tokenizer.decode(summary, skip_special_tokens=True)
    input_text = tokenizer.decode(prompt_ids, skip_special_tokens=True)
    output = decoded[len(input_text):].strip()

    output = re.sub(r'(?i)(summary:|the summary is:|here\'s a summary:|step-by-step|instructions:|note:|###)', '', output)
    output = re.sub(r'^\d+\.[\s]*|\*[\s]*', '', output, flags=re.MULTILINE)
    output = re.sub(r'<\|im_end\|>.*', '', output, flags=re.MULTILINE)
    output = re.sub(r'<\|im_start\|>.*', '', output, flags=re.DOTALL)

    if output and output[-1] not in ['.', '!', '?']:
        last_end = max(output.rfind('.'), output.rfind('!'), output.rfind('?'))
        if last_end > 0:
            output = output[:last_end + 1]
    return output.strip()

# === Generate Summaries ===
def generate_summaries(model, tokenizer, texts, device):
    summaries = []
    for text in tqdm(texts, desc="Generating summaries"):
        prompt = build_prompt(text)
        tokenized = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=2048).to(device)

        output = model.generate(
            **tokenized,
            max_new_tokens=200,
            min_new_tokens=20,
            temperature=0.1,
            top_p=0.5,
            do_sample=True,  # Changed to True to avoid warnings
            early_stopping=False,  # Changed to False to avoid warnings
            repetition_penalty=1.0,
            length_penalty=1.0,
            use_cache=True
        )

        cleaned = clean_summary(output[0], tokenized.input_ids[0], tokenizer)
        summaries.append(cleaned)
    return summaries

# === Evaluation Metrics ===
rouge = evaluate.load("rouge")

from nltk.tokenize import word_tokenize

def compute_bleu_meteor(preds, refs):
    bleu_scores = []
    meteor_scores = []
    for pred, ref in zip(preds, refs):
        pred_tokens = word_tokenize(pred.lower())
        ref_tokens = word_tokenize(ref.lower())
        
        # BLEU expects tokenized input
        bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=SmoothingFunction().method1)
        
        # METEOR also expects tokenized input
        meteor = meteor_score([ref_tokens], pred_tokens)
        
        bleu_scores.append(bleu)
        meteor_scores.append(meteor)
    return bleu_scores, meteor_scores


# === Run Everything ===
generated_a = generate_summaries(model_a, tokenizer_a, input_texts, device="cuda")
generated_b = generate_summaries(model_b, tokenizer_b, input_texts, device="cuda")

rouge_a = rouge.compute(predictions=generated_a, references=reference_summaries)
rouge_b = rouge.compute(predictions=generated_b, references=reference_summaries)

bleu_a, meteor_a = compute_bleu_meteor(generated_a, reference_summaries)
bleu_b, meteor_b = compute_bleu_meteor(generated_b, reference_summaries)

P_a, R_a, F1_a = bert_score(generated_a, reference_summaries, lang="en", rescale_with_baseline=True)
P_b, R_b, F1_b = bert_score(generated_b, reference_summaries, lang="en", rescale_with_baseline=True)

# === Print Results ===
def print_results(name, rouge_dict, bleu_list, meteor_list, f1_tensor):
    # Calculate average for BLEU and METEOR scores (which are lists)
    avg_bleu = np.mean(bleu_list)
    avg_meteor = np.mean(meteor_list)
    
    print(f"\n=== {name} ===")
    print(f"ROUGE-1: {rouge_dict['rouge1']:.4f}")
    print(f"ROUGE-2: {rouge_dict['rouge2']:.4f}")
    print(f"ROUGE-L: {rouge_dict['rougeL']:.4f}")
    print(f"BLEU:    {avg_bleu:.4f}")
    print(f"METEOR:  {avg_meteor:.4f}")
    print(f"BERTScore (F1): {f1_tensor.mean().item():.4f}")

print_results("Model A (Mistral-7B)", rouge_a, bleu_a, meteor_a, F1_a)
print_results("Model B (LLaMA3-1B)", rouge_b, bleu_b, meteor_b, F1_b)

Generating summaries: 100%|██████████| 5/5 [02:11<00:00, 26.21s/it]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Model A (Mistral-7B) ===
ROUGE-1: 0.3505
ROUGE-2: 0.1710
ROUGE-L: 0.2394
BLEU:    0.0913
METEOR:  0.3948
BERTScore (F1): 0.2017

=== Model B (LLaMA3-1B) ===
ROUGE-1: 0.3923
ROUGE-2: 0.2072
ROUGE-L: 0.2874
BLEU:    0.1166
METEOR:  0.4538
BERTScore (F1): 0.2755


In [14]:
# === Ensure tokenizer is set up properly ===
def prepare_model(model, tokenizer):
    if tokenizer.eos_token is None:
        tokenizer.eos_token = "</s>"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return model

prepare_model(model_a, tokenizer_a)
prepare_model(model_b, tokenizer_b)

# === Prompt Template ===
def build_prompt(script_2):
    return f"""<|im_start|>system
You are a summarization AI. Generate only a 2-3 sentence summary. Always complete your sentences.
<|im_end|>
<|im_start|>user
Summarize this article in 2-3 complete sentences:

{script_2}
<|im_end|>
<|im_start|>assistant
"""

# === Clean Summary Output ===
def clean_summary(summary, prompt_ids, tokenizer):
    decoded = tokenizer.decode(summary, skip_special_tokens=True)
    input_text = tokenizer.decode(prompt_ids, skip_special_tokens=True)
    output = decoded[len(input_text):].strip()

    output = re.sub(r'(?i)(summary:|the summary is:|here\'s a summary:|step-by-step|instructions:|note:|###)', '', output)
    output = re.sub(r'^\d+\.[\s]*|\*[\s]*', '', output, flags=re.MULTILINE)
    output = re.sub(r'<\|im_end\|>.*', '', output, flags=re.MULTILINE)
    output = re.sub(r'<\|im_start\|>.*', '', output, flags=re.DOTALL)

    if output and output[-1] not in ['.', '!', '?']:
        last_end = max(output.rfind('.'), output.rfind('!'), output.rfind('?'))
        if last_end > 0:
            output = output[:last_end + 1]
    return output.strip()

# === Generate Summaries ===
def generate_summaries(model, tokenizer, texts, device):
    summaries = []
    for text in tqdm(texts, desc="Generating summaries"):
        prompt = build_prompt(text)
        tokenized = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=2048).to(device)

        output = model.generate(
            **tokenized,
            max_new_tokens=200,
            min_new_tokens=20,
            temperature=0.1,
            top_p=0.5,
            do_sample=True,  # Changed to True to avoid warnings
            early_stopping=False,  # Changed to False to avoid warnings
            repetition_penalty=1.0,
            length_penalty=1.0,
            use_cache=True
        )

        cleaned = clean_summary(output[0], tokenized.input_ids[0], tokenizer)
        summaries.append(cleaned)
    return summaries

# === Evaluation Metrics ===
rouge = evaluate.load("rouge")

from nltk.tokenize import word_tokenize

def compute_bleu_meteor(preds, refs):
    bleu_scores = []
    meteor_scores = []
    for pred, ref in zip(preds, refs):
        pred_tokens = word_tokenize(pred.lower())
        ref_tokens = word_tokenize(ref.lower())
        
        # BLEU expects tokenized input
        bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=SmoothingFunction().method1)
        
        # METEOR also expects tokenized input
        meteor = meteor_score([ref_tokens], pred_tokens)
        
        bleu_scores.append(bleu)
        meteor_scores.append(meteor)
    return bleu_scores, meteor_scores


# === Run Everything ===
generated_a = generate_summaries(model_a, tokenizer_a, input_texts, device="cuda")
generated_b = generate_summaries(model_b, tokenizer_b, input_texts, device="cuda")

rouge_a = rouge.compute(predictions=generated_a, references=reference_summaries)
rouge_b = rouge.compute(predictions=generated_b, references=reference_summaries)

bleu_a, meteor_a = compute_bleu_meteor(generated_a, reference_summaries)
bleu_b, meteor_b = compute_bleu_meteor(generated_b, reference_summaries)

P_a, R_a, F1_a = bert_score(generated_a, reference_summaries, lang="en", rescale_with_baseline=True)
P_b, R_b, F1_b = bert_score(generated_b, reference_summaries, lang="en", rescale_with_baseline=True)

# === Print Results ===
def print_results(name, rouge_dict, bleu_list, meteor_list, f1_tensor):
    # Calculate average for BLEU and METEOR scores (which are lists)
    avg_bleu = np.mean(bleu_list)
    avg_meteor = np.mean(meteor_list)
    
    print(f"\n=== {name} ===")
    print(f"ROUGE-1: {rouge_dict['rouge1']:.4f}")
    print(f"ROUGE-2: {rouge_dict['rouge2']:.4f}")
    print(f"ROUGE-L: {rouge_dict['rougeL']:.4f}")
    print(f"BLEU:    {avg_bleu:.4f}")
    print(f"METEOR:  {avg_meteor:.4f}")
    print(f"BERTScore (F1): {f1_tensor.mean().item():.4f}")

print_results("Model A (Mistral-7B)", rouge_a, bleu_a, meteor_a, F1_a)
print_results("Model B (LLaMA3-1B)", rouge_b, bleu_b, meteor_b, F1_b)

Generating summaries: 100%|██████████| 50/50 [19:04<00:00, 22.89s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Model A (Mistral-7B) ===
ROUGE-1: 0.3464
ROUGE-2: 0.1596
ROUGE-L: 0.2315
BLEU:    0.0937
METEOR:  0.3782
BERTScore (F1): 0.2120

=== Model B (LLaMA3-1B) ===
ROUGE-1: 0.3608
ROUGE-2: 0.1563
ROUGE-L: 0.2398
BLEU:    0.0952
METEOR:  0.3626
BERTScore (F1): 0.2305
